<br>

# Introdução

*Script* que cria uma tabela que apresenta informações das Agências Ambientais da CETESB que atuam nos 645 município deo Estado de São Paulo.

In [ ]:
#!pip3 install open-geodata --upgrade

In [ ]:
import os
import urllib.parse

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from open_geodata import geo
from paths import *

<br>

# Função

In [ ]:
def get_agencia_ambiental(mun):
    """
    Obtem informações de um dado município
    https://licenciamento.cetesb.sp.gov.br/municipioss.asp?muni=SANTOS

    :param mun: _description_
    :type mun: _type_
    :return: _description_
    :rtype: _type_
    """
    # Município
    mun_url = mun.encode('ISO-8859-1')
    mun_url = urllib.parse.quote(mun_url)
    print(f'{mun} > {mun_url}')
    
    # Set URL
    url = f'https://licenciamento.cetesb.sp.gov.br/municipioss.asp?muni={mun_url}'
    
    # Get Data
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    #time.sleep(1)

    # Get all relevant data
    table = soup.find('table',  attrs={'width': '300'})
    rows = table.find_all('tr')
    
    # Create List Adress
    list_end = [mun]
    for row in rows:
        text = row.get_text()
        text = ' '.join(text.split())
        text = text.strip()
        list_end.append(text)
    
    # Rename List    
    for i in ['Bairro:', 'Cidade:', 'CEP:', 'Fone:', 'Fax:', 'E-mail:']:
        list_end = np.char.replace(list_end, i, '')
    list_end = [x.strip() for x in list_end]
    
    # Format Variables
    cep = '{:08d}'.format(int(list_end[5]))
    tel = list_end[6].strip().replace(' ', '').replace('-', ' ')
    fax = list_end[7].strip().replace(' ', '').replace('-', ' ')
    
    # Create Dictionary
    return {
        'municipio_cetesb' : list_end[0],
        'agencia'          : list_end[1],
        'endereco'         : list_end[2].split('nº')[0].strip(),
        'numero'           : list_end[2].split('nº')[1].strip(),
        'bairro'           : list_end[3],
        'municipio_sede'   : list_end[4].title(),
        'cep'              : '{}-{}'.format(cep[:5], cep[5:8]),
        'telefone'         : '({}) {}-{}'.format(tel[:2], tel[3:7], tel[7:]),
        'fax'              : '({}) {}-{}'.format(fax[:2], fax[3:7], fax[7:]),
        'email'            : list_end[8],
        'url'              : url
    }

Uma vez que a função foi definida, basta fazer um teste. Escolhi o município de Piracicaba, pois não tem acentos e grafias divergentes nas diferentes bases de dados.

In [ ]:
get_agencia_ambiental('Caraguatatuba')

<br>

# *Loop*

Há municípios que apresentam apóstrofo e muitas vezes é escrito como acento ou até mesmo divergência na grafia do nome do município, por exemplo, "Moji Mirim" e "Mogi Mirim". Por isso, optei por ler uma tabela que tem os nomes dos municípios utilizados pela CETESB e, uma vez que fiz isso, basta inserir o nome que a instituição padronizou, por meio de um *loop* (*for*).

In [ ]:
# Read Dataframe
df = pd.read_csv(
    input_path / 'tab_municipios.csv',
)

# Filtrar os 10 primeiros
# TODO: Se filtrar, dá erro depois
#df = df[:10]

# Results
df.head()

In [ ]:
# Create empty list
list_addresses = []

#df = df[0:4]  # For tests only
for index, row in df.iterrows():
    # Create Small Dictionary
    dict_df = {
        'id_municipio': row['id_municipio'],        
    }
    
    # Get Addresses
    list_address = get_agencia_ambiental(row['municipio_cetesb'])
    list_address = {**dict_df, **list_address}
    list_addresses.append(list_address)

<br>

# Tabela Municípios

In [ ]:
# Create Table from dictionarys
df = pd.DataFrame.from_dict(list_addresses)

# Results
df.head()

In [ ]:
df_mun = geo.load_dataset('tab.sp.tab_municipio_nome')

df_mun.info()
df_mun.head()

In [ ]:
# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='id_municipio',
    right_on='id_municipio'
)

# Results
df.head()

In [ ]:
# Split
df[['endereco','complemento']] = df['endereco'].str.split(' - ', expand=True)

# Reorder
cols = df.columns.tolist()
cols = cols[0:5] + [cols[-1]] + cols[5:-1]
df = df[cols]

In [ ]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        # Nome
        'Av ': 'Av. ',
        'Av. ': 'Avenida ',
        ' Joao ': ' João ',
        'Ten.'    : 'Tenente',
        
        # Basics
        ' Com '  : ' com ',
        ' Sobre ': ' sobre ',
        ' Da '   : ' da ',
        ' De '   : ' de ',
        ' Do '   : ' do ',
        ' Das '  : ' das ',
        ' Dos '  : ' dos ',
        ' A '    : ' a ',
        ' As '   : ' as ',
        ' Ao '   : ' ao ',
        ' Aos '  : ' aos ',
        ' E '    : ' e ',
        ' O '    : ' o ',
        ' Os '   : ' os ',
        
        '(11) -' : '',
        '( ) -'  : '',
        
        
        'Jd. '  : 'Jardim ',
        'V. '   : 'Vila ',
        'Sta.': 'Santa',
        'Pq. '   : 'Parque ',
        'Res.': 'Residencial ',
        ' Sao ' : ' São ',
        'J.S. Marcos': 'Jardim São Marcos',
        'Jardim-': 'Jardim -',
        'Xxiii': 'XXIII',
        'Ii': 'II',

        # Empty
        'None' : '',
        'none' : '',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [ ]:
# Adjust Table
df.loc[:, 'endereco'] = df['endereco'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'bairro'] = df['bairro'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'municipio_sede'] = df['municipio_sede'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'fax'] = df['fax'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'complemento'] = df['complemento'].astype(str).apply(lambda x: rename_nome(x))

# Results
df.head()

In [ ]:
#set(df['agencia'])
#set(df['endereco'])
#set(df['numero'])
#set(df['bairro'])
#set(df['municipio_sede'])
#set(df['cep'])
#set(df['telefone'])
#set(df['fax'])
#set(df['email'])

In [ ]:
# Deleta colunas
df.drop(['municipio_cetesb'], axis=1, inplace=True)

# Results
df.head()

In [ ]:
# Save
df.to_csv(
    output_path_tab / 'tab_municipio_cetesb.csv',
    index=False,
)

<br>

# Tabela CETESB

In [ ]:
# Save
df = pd.read_csv(
    output_path_tab / 'tab_municipio_cetesb.csv',
)
df.head()

In [ ]:
# Get column's name
cols = list(df.columns[2:-1])
cols

In [ ]:
# Group
df = pd.DataFrame(df.groupby(cols, dropna=False)['id_municipio'].count())
df.reset_index(inplace=True)

# Results
df.head()

In [ ]:
df.columns[len(df.columns)-1]

In [ ]:
df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)

# Results
df.head()

In [ ]:
# Save
df.to_csv(
    os.path.join(output_path_tab, 'tab_cetesb.csv'),
    index=False,    
)

# Results
df.head()